
# Frage 2 – Notebook 04: Tests

Dieses Notebook prüft:
- **Chi²-Test**: Unterscheidet sich die Verteilung der Zahlungsarten (Umfrage) **signifikant** von Statista 2023?
- (Optional) Einfache Auswertung BNPL-Veränderung (deskriptiv).


## Pfade & Imports

In [1]:

from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

# Projekt-Root
project_root = Path("D:/JupyterProject")
out = project_root / "Frage2"

umf_methods = pd.read_csv(out / "umfrage_methods_norm.csv").set_index("Kategorie")
online = pd.read_csv(out / "online_normalisiert.csv").set_index("Kategorie")


## 1) Chi²-Test – Umfrage vs. Statista 2023

In [2]:

merged = umf_methods.join(online[["pct_2023"]], how="outer").fillna(0)

# Fokus-Kategorien (wie in den Plots)
rename_short = {
    "Überweisung/Online-Transfer":"Überweisung",
    "Lastschrift/SEPA":"Lastschrift",
    "E-Wallet":"E-Wallet (PayPal)"
}
merged.index = merged.index.map(lambda s: rename_short.get(s, s))

focus = ["E-Wallet (PayPal)","Kreditkarte","Rechnung","Lastschrift","Überweisung","Mobile Wallet","BNPL (Klarna)"]
m = merged.reindex(focus).fillna(0)

# Chi² braucht Zählwerte: skaliere Statista-% auf Umfrageumfang N
N = int(umf_methods["Anzahl"].sum())
umf_counts = np.rint(m["Anzahl"].to_numpy())
stat_counts = np.rint(m["pct_2023"].to_numpy() / 100.0 * N)

cont = np.vstack([umf_counts, stat_counts]).astype(float)

# Spalten entfernen, die in beiden Gruppen 0 sind
keep = ~(np.logical_and(cont[0] == 0, cont[1] == 0))
cont = cont[:, keep]

# kleine Korrektur, falls 0-Zellen existieren
if (cont == 0).any():
    cont += 0.5

chi2, p, dof, exp = chi2_contingency(cont)
print(f"Chi²={chi2:.2f}, df={dof}, p={p:.4f}")
if p < 0.05:
    print("→ Verteilungen unterscheiden sich signifikant (5%-Niveau).")
else:
    print("→ Kein signifikanter Unterschied (5%-Niveau).")    


Chi²=9.32, df=6, p=0.1562
→ Kein signifikanter Unterschied (5%-Niveau).
